In [38]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
file_path = pd.read_csv('output_data_weatherpy/cities.csv')

# Store csv created in part one into a DataFrame
df = pd.DataFrame(file_path)
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Atambua,-9.1061,124.8925,78.19,79,94,3.06,ID,1616804091
1,1,Jamestown,42.0970,-79.2353,43.00,93,90,12.30,US,1616804092
2,2,Gigmoto,13.7796,124.3900,81.37,80,100,8.99,PH,1616804092
3,3,Ilulissat,69.2167,-51.1000,-13.00,48,14,7.36,GL,1616804092
4,4,Ushuaia,-54.8000,-68.3000,50.00,62,75,5.75,AR,1616804093


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [46]:
# Heatmap of humidity
locations = df[['Lat','Lng']].astype(float)
humidity = df['Humidity'].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [98]:
# Narrow down cities that fit criteria and drop any results with null values
new_df = df.loc[(df['Max Temp'] < 75) & (df['Humidity'] < 25) & (df['Wind Speed'] < 6),:]
len(new_df)

9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [111]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = new_df[['City','Country','Lat','Lng']].reset_index(drop=True)
hotel_df

,City,Country,Lat,Lng
0,Mahāsamund,IN,21.1000,82.1000
1,Tura,IN,25.5198,90.2201
2,Ukiah,US,39.1502,-123.2078
3,Sayyan,YE,15.1718,44.3244
4,Bedēsa,ET,8.9000,40.7833
5,Susa,IR,32.1942,48.2436
6,Dalbandin,PK,28.8947,64.4101
7,Khuzdar,PK,27.7384,66.6434
8,Gādarwāra,IN,22.9167,78.7833


In [112]:
# Add new column for hotel name
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Mahāsamund,IN,21.1000,82.1000,
1,Tura,IN,25.5198,90.2201,
2,Ukiah,US,39.1502,-123.2078,
3,Sayyan,YE,15.1718,44.3244,
4,Bedēsa,ET,8.9000,40.7833,
5,Susa,IR,32.1942,48.2436,
6,Dalbandin,PK,28.8947,64.4101,
7,Khuzdar,PK,27.7384,66.6434,
8,Gādarwāra,IN,22.9167,78.7833,


In [113]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

# Iterate through 
for index, row in hotel_df.iterrows():

    # get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_search = requests.get(base_url, params=params)
    
    # convert to json
    hotel_search = hotel_search.json()

    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_search['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [114]:
# Drop rows with missing field/result
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Mahāsamund,IN,21.1000,82.1000,hotel Acacia inn
1,Tura,IN,25.5198,90.2201,Hotel Polo Orchid
2,Ukiah,US,39.1502,-123.2078,Hampton Inn Ukiah
4,Bedēsa,ET,8.9000,40.7833,Bank Of Abyssinia
5,Susa,IR,32.1942,48.2436,اقامتگاه سنتی ماهور
6,Dalbandin,PK,28.8947,64.4101,Al-Dawood Hotel & Restaurant
8,Gādarwāra,IN,22.9167,78.7833,Hotel Arjun Palace


In [103]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [106]:
# Add marker layer ontop of heat map
hotel_names = hotel_df['Hotel Name'].tolist()

hotel_marker = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)

# Display figure
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_marker)

fig

Figure(layout=FigureLayout(height='420px'))